In [1]:
import numpy as np
import pickle
import random
import matplotlib.pyplot as plt

In [23]:
# config = {}
# config['layer_specs'] = [784, 100, 100, 10]  # The length of list denotes number of hidden layers; each element denotes number of neurons in that layer; first element is the size of input layer, last element is the size of output layer.
# config['activation'] = 'sigmoid' # Takes values 'sigmoid', 'tanh' or 'ReLU'; denotes activation function for hidden layers
# config['batch_size'] = 1000  # Number of training samples per batch to be passed to network
# config['epochs'] = 50  # Number of epochs to train the model
# config['early_stop'] = True  # Implement early stopping or not
# config['early_stop_epoch'] = 5  # Number of epochs for which validation loss increases to be counted as overfitting
# config['L2_penalty'] = 0  # Regularization constant
# config['momentum'] = False  # Denotes if momentum is to be applied or not
# config['momentum_gamma'] = 0.9  # Denotes the constant 'gamma' in momentum expression
# config['learning_rate'] = 0.0001 # Learning rate of gradient descent algorithm

In [3]:
# softmax activation function that takes in a numpy array and returns a numpy array.
# take in m * n array and returns m * n array
def softmax(x):
  return np.array([[ele/np.sum(lst) for ele in lst] for lst in np.exp(x)])

# read the data and return it as 2 numpy arrays.
# convert labels to one hot encoded format.
def load_data(fname):
  with open(fname, 'rb') as f:
    data = pickle.load(f)
    images = data[:, :-1]
    temp = data[:, -1]  # not yet one-hot encoded label
    labels = np.array([np.zeros(10) for _ in range(len(images))])
    for i in range(len(labels)):
      labels[i][int(temp[i])] = 1.0
  return images, labels

In [4]:
class Activation:
  def __init__(self, activation_type = "sigmoid"):
    self.activation_type = activation_type
    self.x = None # Save the input 'x' for sigmoid or tanh or ReLU to this variable since it will be used later for computing gradients.
  
  def forward_pass(self, a):
    if self.activation_type == "sigmoid":
      return self.sigmoid(a)
    
    elif self.activation_type == "tanh":
      return self.tanh(a)
    
    elif self.activation_type == "ReLU":
      return self.ReLU(a)
  
  def backward_pass(self, delta):
    if self.activation_type == "sigmoid":
      grad = self.grad_sigmoid()
    
    elif self.activation_type == "tanh":
      grad = self.grad_tanh()
    
    elif self.activation_type == "ReLU":
      grad = self.grad_ReLU()
    
    return grad * delta
 
  # sigmoid activation function that takes in a numpy array and returns a numpy array      
  def sigmoid(self, x):
    self.x = x
    output = 1 / (1+np.exp(-x))
    return output

  # tanh activation function that takes in a numpy array and returns a numpy array
  def tanh(self, x):
    self.x = x
    output = np.tanh(x)
    return output

  # ReLU activation function that takes in a numpy array and returns a numpy array
  def ReLU(self, x):
    self.x = x
    output = np.maximum(0, x)
    return output

  # gradient through sigmoid activation function that takes in a numpy array and returns a numpy array
  def grad_sigmoid(self):
    grad = np.exp(-self.x)/np.power((1+np.exp(-self.x)), 2)
    return grad

  # gradient through tanh activation function that takes in a numpy array and returns a numpy array
  def grad_tanh(self):
    grad = 1-np.power(np.tanh(self.x), 2)
    return grad

  # gradient through ReLU activation function that takes in a numpy array and returns a numpy array
  def grad_ReLU(self):
    grad = np.copy(self.x)
    grad[grad<=0] = 0
    grad[grad>0] = 1
    return grad

In [5]:
class Layer():
  def __init__(self, in_units, out_units):
    np.random.seed(42)
    self.w = np.random.randn(in_units, out_units)  # Weight matrix
    self.b = np.zeros((1, out_units)).astype(np.float32)  # Bias
    self.b_in = None
    self.x = None  # Save the input to forward_pass in this
    self.a = None  # Save the output of forward pass in this (without activation)
    self.d_x = None  # Save the gradient w.r.t x in this
    self.d_w = None  # Save the gradient w.r.t w in this
    self.d_b = None  # Save the gradient w.r.t b in this
    self.delta_w = np.zeros((in_units, out_units))  # keep track of w difference for momentum
    self.best_w = None # save the best weight for this layer

  # forward pass through a layer. Do not apply activation function here
  # if x is N * M, then b_in should be N * 1
  # w should be M * K, b should be 1 * K
  def forward_pass(self, x):
    self.x = x 
    self.b_in = np.ones((len(self.x),1))
    self.a = np.concatenate((self.b_in, self.x), axis=1) @ \
             np.concatenate((self.b, self.w), axis=0)
    return self.a
  
  # backward pass. This takes in gradient from its next layer as input,
  # computes gradient for its weights and the delta to pass to its previous layers.
  def backward_pass(self, delta):
    self.d_w = self.x.T @ delta
    self.d_x = delta @ self.w.T
    self.d_b = self.b_in.T @ delta
    return self.d_x

In [44]:
class Neuralnetwork():
  def __init__(self, config):
    self.layers = []
    self.x = None  # Save the input to forward_pass in this
    self.y = None  # Save the output vector of model in this
    self.targets = None  # Save the targets in forward_pass in this variable
    for i in range(len(config['layer_specs']) - 1):
      self.layers.append( Layer(config['layer_specs'][i], config['layer_specs'][i+1]) )
      if i < len(config['layer_specs']) - 2:
        self.layers.append(Activation(config['activation']))  
  
  # forward pass through all layers of the model and return loss and predictions.
  # If targets == None, loss should be None. If not, then return the loss computed.
  def forward_pass(self, x, targets=None):
    self.x = x
    self.targets = targets
    a = np.copy(x)
    for layer in self.layers:
      a = layer.forward_pass(a)
    self.y = softmax(a)
    loss = None if np.array_equal(self.targets, self.y) else self.loss_func(self.y, self.targets)
    accuracy = self.accuracy_func(self.y, self.targets)
    return loss, self.y, accuracy


  # find cross entropy loss between logits and targets
  def loss_func(self, logits, targets):
    loss = - np.sum(targets * np.log(logits))/len(logits)
    return loss

  def accuracy_func(self, logits, targets):
    accurate = 0
    for (logit, target) in zip(logits, targets):
        p = np.argmax(logit)
        t = np.argmax(target)
        accurate += (1 if p==t else 0)
    return accurate/len(logits)

  # implement the backward pass for the whole network. 
  def backward_pass(self):
    idx = len(self.layers)-1
    delta = self.targets - self.y
    for layer in reversed(self.layers):
      delta = layer.backward_pass(delta)

In [45]:
# train the network. Use values from config to set parameters
# such as L2 penalty, number of epochs, momentum, etc.
def trainer(model, X_train, y_train, X_valid, y_valid, config):
  best_holdout_loss = float('inf')  # used to keep track of the best weights
  last_holdout_loss = float('inf')    # last holdout_loss, used for early stopping
  num_increase = 0  # consecutive number of times holdout_loss goes up, used for early stopping

  plot_train_loss = []
  plot_train_accuracy = []
  plot_holdout_loss = []
  plot_holdout_accuracy = []

  for epoch in range(config['epochs']):
    print(epoch, end=" ")
    
    # randomnize the data and split them into batches
    idx = [i for i in range(len(X_train))]
    random.shuffle(idx)
    num_batches = int(len(X_train)/config['batch_size'])
    n = 0
    
    net_train_loss = 0
    net_train_accuracy = 0  
    
    # put each batch through the neural network
    for _ in range(num_batches):
      x = X_train[n:n+config['batch_size']]
      y = y_train[n:n+config['batch_size']]
      n += config['batch_size']

      # pass the batch through the neuro network
      train_loss, train_predicts, train_accuracy = model.forward_pass(x, y)
      model.backward_pass()

      net_train_loss += train_loss
      net_train_accuracy += train_accuracy
        
      # update weights for each layer
      for layer in model.layers[::2]:
        new_delta_w = config['learning_rate'] * layer.d_w
        if config['momentum']: 
          new_delta_w += config['momentum_gamma'] * layer.delta_w
          layer.delta_w = new_delta_w
        layer.w += new_delta_w

    # all batches done. calculate validation error 
    holdout_loss, holdout_predicts, holdout_accuracy = model.forward_pass(X_valid, y_valid)
    
    # accumulate data used for plotting later
    plot_train_loss.append(net_train_loss/num_batches)
    plot_train_accuracy.append(round(net_train_accuracy*100/num_batches, 2))
    plot_holdout_loss.append(holdout_loss)
    plot_holdout_accuracy.append(round(holdout_accuracy*100, 2))

    # update best weight for each layer
    if holdout_loss < best_holdout_loss:
      best_holdout_loss = holdout_loss
      for layer in model.layers[::2]:
        layer.best_w = layer.w

    # check if we should early stop training
    if config['early_stop']:
      if holdout_loss >= last_holdout_loss:
        num_increase += 1
        if num_increase == config['early_stop_epoch']:
          print("early stopped!!!")
          break
      else:
        num_increase = 0
      last_holdout_loss = holdout_loss
    
  print()
  return plot_train_loss, plot_train_accuracy, plot_holdout_loss, plot_holdout_accuracy

In [46]:
# run the model on the data passed as input and return accuracy
def test(model, X_test, y_test, config):
  for layer in model.layers[::2]:
    layer.w = layer.best_w
  test_loss, test_predicts, test_accuracy = model.forward_pass(X_test, y_test)    
  return test_accuracy

In [47]:
def train_with_config(config, X_train, y_train, X_valid, y_valid, X_test, y_test):
  print("lr:", config['learning_rate'],'momentum:',config['momentum'],'momentum_gamma:',config['momentum_gamma'],\
        "activation:",config['activation'],'layer_specs:',config['layer_specs'])
    
  model = Neuralnetwork(config)

  t_loss, t_acc, h_loss, h_acc = trainer(model, X_train, y_train, X_valid, y_valid, config)
  epoch = [int(i+1) for i in range(len(t_loss))]
  test_acc = test(model, X_test, y_test, config)
  print("test accuracy:", test_acc)
  plt.plot(epoch, t_loss)
  plt.plot(epoch, h_loss)
  plt.legend(['train loss', 'holdout loss'], loc='center left', bbox_to_anchor=(1, 0.5))
  plt.xlabel('epoch number')
  plt.ylabel('train/holdout loss')
  plt.show()
  plt.plot(epoch, t_acc)
  plt.plot(epoch, h_acc)
  plt.legend(['train accuracy', 'holdout accuracy'], loc='center left', bbox_to_anchor=(1, 0.5))
  plt.xlabel('epoch number')
  plt.ylabel('train/holdout accuracy in percentage')
  plt.show()

In [ ]:
if __name__ == "__main__":
  train_data_fname = './data/MNIST_train.pkl'
  valid_data_fname = './data/MNIST_valid.pkl'
  test_data_fname = './data/MNIST_test.pkl'
  X_train, y_train = load_data(train_data_fname)
  X_valid, y_valid = load_data(valid_data_fname)
  X_test, y_test = load_data(test_data_fname)

  config = {}
  config['layer_specs'] = [784, 50, 10]  
  config['activation'] = 'sigmoid'
  config['batch_size'] = 1000
  config['epochs'] = 50
  config['early_stop'] = True 
  config['early_stop_epoch'] = 5 
  config['L2_penalty'] = 0
  config['momentum'] = True
  config['momentum_gamma'] = 0.9 
  config['learning_rate'] = 0.00005


  train_with_config(config, X_train, y_train, X_valid, y_valid, X_test, y_test)
    

lr: 5e-05 momentum: True momentum_gamma: 0.9 activation: sigmoid layer_specs: [784, 50, 10]
0 1 